## Market Basket Analysis
#### by **Ivan Alducin**
<p><img src="https://miro.medium.com/max/757/1*UxsrvB1oWpTYxUgSRNW92g.jpeg" width="1000"/></p>

## Introducción
<p>En este capítulo estudiaremos uno de los algoritmos de <i>Machine Learning</i> que mas uso tiene dentro de la industria de retail, y este es el <b>Market Basket Analysis</b>, el cual detecta los patrones de compra de los clientes en el cual básicamente nos dice que la compra de algunos productos afectará la compra de otros produtos. Se puede utiizar para predecir productos de interés en los clientes.

Para este caso de estudio regresaremos a nuestro conjunto de datos de <code>Online Retail</code></p>

In [1]:
# Importa Pandas
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Importa el archivo "Online Retail.csv"
df = pd.read_csv('Online Retail.csv', encoding='latin1')
df.head()

,INVOICE_NO,STOCK_CODE,DESCRIPTION,QUANTITY,INVOICE_DATE,UNIT_PRICE,CUSTOMER_ID,REGION
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01/12/2019 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01/12/2019 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01/12/2019 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01/12/2019 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01/12/2019 08:26,3.39,17850.0,United Kingdom


In [10]:
# Análisis Exploratorio
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   INVOICE_NO    541909 non-null  object 
 1   STOCK_CODE    541909 non-null  object 
 2   DESCRIPTION   540455 non-null  object 
 3   QUANTITY      541909 non-null  int64  
 4   INVOICE_DATE  541909 non-null  object 
 5   UNIT_PRICE    541909 non-null  float64
 6   CUSTOMER_ID   406829 non-null  float64
 7   REGION        541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


## Support, Confidende and Lift
<p>Para entener como funciona el algoritmo, hay que entender una serie de conceptos:

- <b>Support</b>: Calcula la frecuencia con la que se compra el producto
- <b>Confidence</b>: Mide la frecuencia con la que aparecen elementos de <b>Y</b> en transacciones que contienen <b>X</b>
- <b>Lift</b>: Es el valor que nos dice la probabilidad de que se compre el artículo <b>Y</b> junto con el artículo <b>X</b>. Cuantifica una regla para predecir la compra que simplemente asumir el resultado en primer lugar. Los valores mayores a uno indican que es probable que los artículos se compren juntos</p>
    
<p><img src="https://miro.medium.com/max/2134/1*--iUPe_DtzKdongjqZ2lOg.png", width=1050 =/></p>

## Pre-Processing
<p>Previo a construir el modelo, debemos depurar los valores <code>NaN</code> o Nulos en nuestros conjuntos de datos. Ya sabemos que eliminar todos los valores nulos no es la mejor práctica, pero en esta ocasión y para simplificar las cosas lo haremos así ;)

También fitraremos las compras con cantidades positivas, ya que al ser estar trabajando datos de una tienda en línea, recordemos que puede haber devoluciones, por lo que seguramente hay cantidades negativas. Hay que asegurarnos de no tomarlas en cuenta</p>

In [2]:
# Eliminar los valores nulos
df = df.dropna()

In [3]:
# Filtrar las compras con una cantidad mayor o igual a 0
df = df[df['QUANTITY'] > 0]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 397924 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   INVOICE_NO    397924 non-null  object 
 1   STOCK_CODE    397924 non-null  object 
 2   DESCRIPTION   397924 non-null  object 
 3   QUANTITY      397924 non-null  int64  
 4   INVOICE_DATE  397924 non-null  object 
 5   UNIT_PRICE    397924 non-null  float64
 6   CUSTOMER_ID   397924 non-null  float64
 7   REGION        397924 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 27.3+ MB


In [13]:
df['REGION'].value_counts(normalize=True) * 100

United Kingdom          89.048411
Germany                  2.272293
France                   2.096380
EIRE                     1.818940
Spain                    0.624491
Netherlands              0.593832
Belgium                  0.510399
Switzerland              0.462902
Portugal                 0.367407
Australia                0.297796
Norway                   0.269398
Italy                    0.190489
Channel Islands          0.187976
Finland                  0.172143
Cyprus                   0.154301
Sweden                   0.113338
Austria                  0.100019
Denmark                  0.095496
Poland                   0.082930
Japan                    0.080669
Israel                   0.062323
Unspecified              0.061318
Singapore                0.055790
Iceland                  0.045737
USA                      0.044983
Canada                   0.037947
Greece                   0.036439
Malta                    0.028146
United Arab Emirates     0.017089
European Commu

## Canasta
<p>Una canasta es la compra por factura. Los datos de cada canasta deben contener  la cantidad de cada artículo comprado por transacción (número de factura). ¿Cómo hacemos eso?</p>

In [4]:
product = df[df['REGION'] == 'United Kingdom'].groupby(['INVOICE_NO', 'DESCRIPTION'])['QUANTITY'].sum()
product

INVOICE_NO  DESCRIPTION                        
536365      CREAM CUPID HEARTS COAT HANGER          8
            GLASS STAR FROSTED T-LIGHT HOLDER       6
            KNITTED UNION FLAG HOT WATER BOTTLE     6
            RED WOOLLY HOTTIE WHITE HEART.          6
            SET 7 BABUSHKA NESTING BOXES            2
                                                   ..
581585      ZINC WILLIE WINKIE  CANDLE STICK       24
581586      DOORMAT RED RETROSPOT                  10
            LARGE CAKE STAND  HANGING STRAWBERY     8
            RED RETROSPOT ROUND CAKE TINS          24
            SET OF 3 HANGING OWLS OLLIE BEAK       24
Name: QUANTITY, Length: 344362, dtype: int64

In [5]:
# Calcula el total de productos por factura (Solo para Reino Unido)
product = df[df['REGION'] == 'United Kingdom'].groupby(['INVOICE_NO', 'DESCRIPTION'])['QUANTITY'].sum()


# Agrupa los productos que se vendieron por factura, usaremos la funcion .unstack()
basket = product.unstack().reset_index().fillna(0).set_index('INVOICE_NO')
basket.head()

DESCRIPTION,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
INVOICE_NO,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536366,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536368,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
product = df[df['REGION'] == 'United Kingdom'].groupby(['INVOICE_NO', 'DESCRIPTION']).agg({'QUANTITY':'sum'})
product

QUANTITY
INVOICE_NO DESCRIPTION                                  
536365     CREAM CUPID HEARTS COAT HANGER              8
           GLASS STAR FROSTED T-LIGHT HOLDER           6
           KNITTED UNION FLAG HOT WATER BOTTLE         6
           RED WOOLLY HOTTIE WHITE HEART.              6
           SET 7 BABUSHKA NESTING BOXES                2
...                                                  ...
581585     ZINC WILLIE WINKIE  CANDLE STICK           24
581586     DOORMAT RED RETROSPOT                      10
           LARGE CAKE STAND  HANGING STRAWBERY         8
           RED RETROSPOT ROUND CAKE TINS              24
           SET OF 3 HANGING OWLS OLLIE BEAK           24

[344362 rows x 1 columns]

In [13]:
basket = product.pivot_table(index='INVOICE_NO', 
                             columns='DESCRIPTION', 
                             values='QUANTITY',
                             fill_value=0)
basket

DESCRIPTION,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
INVOICE_NO,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
# Vamos a binarizar (One-Hot encoding) los productos por factura, es decir 1 si tiene algún producto en la compra, 0 si no tiene
def encode_units(x):
    if(x <= 0):
        r = 0
    if(x > 0):
        r = 1
    return r

encode = basket.applymap(encode_units)
encode.head()

DESCRIPTION,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
INVOICE_NO,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536369,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# Vamos a tomar solo las compras con dos productos o mas
final_basket = encode[(encode > 0).sum(axis=1) > 1]
final_basket

DESCRIPTION,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TOADSTOOL BEDSIDE LIGHT,...,ZINC STAR T-LIGHT HOLDER,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
INVOICE_NO,,,,,,,,,,,,,,,,,,,,,
536365,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536366,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536368,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536372,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Algoritmo Apriori
<p>El algoritmo <i>Apriori</i> utiliza los conceptos vistos anteriormente para poder crear las reglas de asociación, si quieres saber un poco más de detalle acerca de este algorimo puedes visitar el siguiente <a href="https://www-users.cs.umn.edu/~kumar001/dmbook/ch5_association_analysis.pdf">link</a></p>

<p><img src="https://miro.medium.com/max/700/1*JCMoMh8qeXPTwBWlF8ccuA.png", width=750 =/></p>

In [17]:
# Importar modulos necesarios
from mlxtend.frequent_patterns import apriori

# Aplicar el algoritmo apriori
rules = apriori(final_basket, 
                         min_support=0.03,
                         use_colnames=True).sort_values('support', ascending=False).reset_index(drop=True)

## Reglas de asociación
<p>Al aplicar el algoritmo a priori, podemos definir la frecuencia de los datos dando el valor de soporte. En este caso, definimos artículos comprados con frecuencia como artículos que se compran hasta en un 3% de toda la transacción.</p>

In [18]:
# Importar modulos necesarios
from mlxtend.frequent_patterns import association_rules

# Crear las Reglas de Aosciación
association_rules(rules, metric='lift', min_threshold=1).sort_values('lift', ascending=False).reset_index(drop=True)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.039802,0.043900,0.030957,0.777778,17.717202,0.029210,4.302452
1,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.043900,0.039802,0.030957,0.705185,17.717202,0.029210,3.256952
2,(LUNCH BAG RED RETROSPOT),(LUNCH BAG PINK POLKADOT),0.072841,0.055086,0.030632,0.420536,7.634188,0.026620,1.630668
3,(LUNCH BAG PINK POLKADOT),(LUNCH BAG RED RETROSPOT),0.055086,0.072841,0.030632,0.556080,7.634188,0.026620,2.088574
4,(JUMBO BAG RED RETROSPOT),(JUMBO BAG PINK POLKADOT),0.093197,0.052680,0.032908,0.353105,6.702899,0.027999,1.464412
5,(JUMBO BAG PINK POLKADOT),(JUMBO BAG RED RETROSPOT),0.052680,0.093197,0.032908,0.624691,6.702899,0.027999,2.416152
6,(LUNCH BAG BLACK SKULL.),(LUNCH BAG RED RETROSPOT),0.064646,0.072841,0.031478,0.486922,6.684737,0.026769,1.807051
7,(LUNCH BAG RED RETROSPOT),(LUNCH BAG BLACK SKULL.),0.072841,0.064646,0.031478,0.432143,6.684737,0.026769,1.647164
